In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import normalize
from sklearn import datasets, linear_model
import hdbscan
import seaborn as sns
import statsmodels.api as sm
from numpy import linalg
from math import sqrt
import time
from sklearn.decomposition import PCA
from skopt import gp_minimize, dump, load
from skopt.space import Real, Integer
from skopt.plots import plot_convergence
from skopt.utils import use_named_args
%matplotlib inline

In [27]:
dataset = pd.read_csv('shopitem.csv')
Shopgroup = pd.read_csv('Shopgroup.csv')
Itemgroup = pd.read_csv('Itemgroup.csv')

In [28]:
dataset=dataset[dataset.item_cnt_month<100]

In [29]:
sparsity=1-dataset.shape[0]/214200/60
sparsity

0.9670021786492374

In [30]:
Ntr=int(dataset.shape[0]*0.6)
Nv=int(dataset.shape[0]*0.8)
sim_data=dataset.values
n=22170
m=60
train=sim_data[:Ntr,:]
validation=sim_data[Ntr:Nv,:]
test=sim_data[Nv:,:]
x_train=train[:,:2]  
x_test=test[:,:2]
x_valid=validation[:,:2]
x_train=x_train.astype(int)
x_test=x_test.astype(int)
x_valid=x_valid.astype(int)
y=train[:,2]
y_test=test[:,2]
y_valid=validation[:,2]

In [31]:
#player set
ind1=[] #matched played for each user
y1=[]
#match set
ind2=[] #players in this match
y2=[]
for u in range(n):
    ind1.append(x_train[x_train[:,0]==u,1])
    y1.append(train[x_train[:,0]==u,2])
for i in range(m):
    ind2.append(x_train[x_train[:,1]==i,0])
    y2.append(train[x_train[:,1]==i,2])

In [32]:
def shrink(x,l):
    if x>l/2:
        X=x-l/2
    elif x<-l/2:
        X=x+l/2
    else:
        X=0
    return X
Vshrink= np.vectorize(shrink)

In [33]:
def mylasso(y,x,k,l,L):
    betaols=linalg.solve(x.transpose()@x+L*np.identity(k),x.transpose()@y)
    beta = Vshrink(betaols,l)
    return beta

In [34]:
def myl0(y,x,k,l,L):
    beta=linalg.solve(x.transpose()@x+L*np.identity(k),x.transpose()@y)
    beta = beta*((beta>np.median(beta))+1)
    return beta

In [35]:
def mygsm(foo,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,matchgroup,playergroup,n,m,k,lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb): 
    inM=matchgroup.In.max()
    exM=matchgroup.Ex.max()
    inN=playergroup.In.max()
    exN=playergroup.Ex.max()
    P=np.random.normal(0,0.1,(n,k))
    Q=np.random.normal(0,0.1,(m,k))
    S=np.random.normal(0,0.1,(n,k))
    T=np.random.normal(0,0.1,(m,k))
    A=np.random.normal(0,0.1,(n,k))
    B=np.random.normal(0,0.1,(m,k))
    Pnew=np.zeros(shape=(n,k))
    Qnew=np.zeros(shape=(m,k))
    Snew=np.zeros(shape=(n,k))
    Tnew=np.zeros(shape=(m,k))
    Anew=np.zeros(shape=(n,k))
    Bnew=np.zeros(shape=(m,k))
    it=1  #number of iterations
    diff=1  #improvement over last iteration
    diffQTB=1
    diffPSA=1
    while(diff>1e-5 or it<10):
        diff=np.sum(np.multiply(Pnew+Snew+Anew-P-S-A,Pnew+Snew+Anew-P-S-A))/n/k+np.sum(np.multiply(Qnew+Tnew+Bnew-Q-T-B,Qnew+Tnew+Bnew-Q-T-B))/m/k
        while(diffQTB>1e-8):
            for i in range(m):
                xpsa=P[ind2[i],:]+S[ind2[i],:]+A[ind2[i],:]
                r=y2[i]-(P[ind2[i],:]+S[ind2[i],:]+A[ind2[i],:])@(T[i,:]+B[i,:])
                Qnew[i,:]=foo(r,xpsa,k,lq,Lq)
            for i in range(inM):
                I=matchgroup.matchId[matchgroup.In==i]
                I=I[I<len(ind2)]
                playerI=[]
                for j in I:
                    playerI=np.concatenate([playerI,ind2[j]]).astype("int")
                xpsa=P[playerI,:]+S[playerI,:]+A[playerI,:]
                r=[]
                for j in I:
                    R=y2[j]-(P[ind2[j],:]+S[ind2[j],:]+A[ind2[j],:])@(Qnew[j,:]+B[j,:])
                    r=np.concatenate([r,R])
                Tnew[I,:]=foo(r,xpsa,k,lt,Lt)
            for i in range(exM):
                I=matchgroup.matchId[matchgroup.Ex==i]
                I=I[I<len(ind2)]
                playerI=[]
                for j in I:
                    playerI=np.concatenate([playerI,ind2[j]]).astype("int")
                xpsa=P[playerI,:]+S[playerI,:]+A[playerI,:]
                r=[]
                for j in I:
                    R=y2[j]-(P[ind2[j],:]+S[ind2[j],:]+A[ind2[j],:])@(Qnew[j,:]+Tnew[j,:])
                    r=np.concatenate([r,R])
                Bnew[I,:]=foo(r,xpsa,k,lb,Lb)
            diffQTB=np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k+np.sum(np.multiply(Tnew-T,Tnew-T))/inM/k+np.sum(np.multiply(Bnew-B,Bnew-B))/exM/k
            Q=Qnew
            T=Tnew
            B=Bnew
        while(diffPSA>1e-8):
            for i in range(n):
                xqtb=Q[ind1[i],:]+T[ind1[i],:]+B[ind1[i],:]
                r=y1[i]-(Q[ind1[i],:]+T[ind1[i],:]+B[ind1[i],:])@(S[i,:]+A[i,:])
                Pnew[i,:]=foo(r,xqtb,k,lp,Lp)

            for i in range(inN):
                I=playergroup.playerId[playergroup.In==i]
                I=I[I<len(ind1)]
                matchI=[]
                for j in I:
                    matchI=np.concatenate([matchI,ind1[j]]).astype("int")
                xqtb=Q[matchI,:]+T[matchI,:]+B[matchI,:]
                r=[]
                for j in I:
                    R=y1[j]-(Q[ind1[j],:]+T[ind1[j],:]+B[ind1[j],:])@(Pnew[j,:]+A[j,:])
                    r=np.concatenate([r,R])
                Snew[I,:]=foo(r,xqtb,k,ls,Ls)

            for i in range(exN):
                I=playergroup.playerId[playergroup.Ex==i]
                I=I[I<len(ind1)]
                matchI=[]
                for j in I:
                    matchI=np.concatenate([matchI,ind1[j]]).astype("int")
                xqtb=Q[matchI,:]+T[matchI,:]+B[matchI,:]
                r=[]
                for j in I:
                    R=y1[j]-(Q[ind1[j],:]+T[ind1[j],:]+B[ind1[j],:])@(Pnew[j,:]+Snew[j,:])
                    r=np.concatenate([r,R])
                Anew[I,:]=foo(r,xqtb,k,la,La)  
            diffPSA=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Snew-S,Snew-S))/inN/k+np.sum(np.multiply(Anew-A,Anew-A))/exN/k
            P=Pnew
            S=Snew
            A=Anew
        it=it+1
    yhat_valid=np.sum(np.multiply((P[x_valid[:,0],:]+S[x_valid[:,0],:]+A[x_valid[:,0],:]),(Q[x_valid[:,1],:]+T[x_valid[:,1],:]+B[x_valid[:,1],:])),1)
    RMSE=sqrt((y_valid-yhat_valid)@(y_valid-yhat_valid)/y_valid.size)
    return RMSE,yhat_valid

In [36]:
def mygsm1(foo,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,matchgroup,playergroup,n,m,k,lp,ls,lq,lt,Lp,Ls,Lq,Lt): 
    inM=matchgroup.In.max()
    inN=playergroup.In.max()
    P=np.random.normal(0,0.1,(n,k))
    Q=np.random.normal(0,0.1,(m,k))
    S=np.random.normal(0,0.1,(n,k))
    T=np.random.normal(0,0.1,(m,k))
    Pnew=np.zeros(shape=(n,k))
    Qnew=np.zeros(shape=(m,k))
    Snew=np.zeros(shape=(n,k))
    Tnew=np.zeros(shape=(m,k))
    yhat=np.sum(np.multiply((P[x_train[:,0],:]+S[x_train[:,0],:]),(Q[x_train[:,1],:]+T[x_train[:,1],:])),1)
    it=1  #number of iterations
    diff=1  #improvement over last iteration
    diffQTB=1
    diffPSA=1
    while(diff>1e-5 or it<10):
        diff=np.sum(np.multiply(Pnew+Snew-P-S,Pnew+Snew-P-S))/n/k+np.sum(np.multiply(Qnew+Tnew-Q-T,Qnew+Tnew-Q-T))/m/k
        while(diffQTB>1e-8):
            for i in range(m):
                xpsa=P[ind2[i],:]+S[ind2[i],:]
                r=y2[i]-(P[ind2[i],:]+S[ind2[i],:])@(T[i,:])
                Qnew[i,:]=foo(r,xpsa,k,lq,Lq)
            for i in range(inM):
                I=matchgroup.matchId[matchgroup.In==i]
                I=I[I<len(ind2)]
                playerI=[]
                for j in I:
                    playerI=np.concatenate([playerI,ind2[j]]).astype("int")
                xpsa=P[playerI,:]+S[playerI,:]
                r=[]
                for j in I:
                    R=y2[j]-(P[ind2[j],:]+S[ind2[j],:])@(Qnew[j,:])
                    r=np.concatenate([r,R])
                Tnew[I,:]=foo(r,xpsa,k,lt,Lt)
            diffQTB=np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k+np.sum(np.multiply(Tnew-T,Tnew-T))/inM/k
            Q=Qnew
            T=Tnew
        while(diffPSA>1e-8):
            for i in range(n):
                xqtb=Q[ind1[i],:]+T[ind1[i],:]
                r=y1[i]-(Q[ind1[i],:]+T[ind1[i],:])@(S[i,:])
                Pnew[i,:]=foo(r,xqtb,k,lp,Lp)
            for i in range(inN):
                I=playergroup.playerId[playergroup.In==i]
                I=I[I<len(ind1)]
                matchI=[]
                for j in I:
                    matchI=np.concatenate([matchI,ind1[j]]).astype("int")
                xqtb=Q[matchI,:]+T[matchI,:]
                r=[]
                for j in I:
                    R=y1[j]-(Q[ind1[j],:]+T[ind1[j],:])@(Pnew[j,:])
                    r=np.concatenate([r,R])
                Snew[I,:]=foo(r,xqtb,k,ls,Ls)
            diffPSA=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Snew-S,Snew-S))/inN/k
            P=Pnew
            S=Snew
        it=it+1
    yhat_valid=np.sum(np.multiply((P[x_valid[:,0],:]+S[x_valid[:,0],:]),(Q[x_valid[:,1],:]+T[x_valid[:,1],:])),1)
    RMSE=sqrt((y_valid-yhat_valid)@(y_valid-yhat_valid)/y_valid.size)
    return RMSE,yhat_valid

In [37]:
def mygsm0(foo,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,n,m,k,lp,lq,Lp,Lq): 
    P=np.random.normal(2,1,(n,k))
    Q=np.random.normal(2,1,(m,k))
    Pnew=np.zeros(shape=(n,k))
    Qnew=np.zeros(shape=(m,k))
    yhat=np.sum(np.multiply((P[x_train[:,0],:]),(Q[x_train[:,1],:])),1)
    it=1  #number of iterations
    diff=1  #improvement over last iteration
    diffQTB=1
    diffPSA=1
    while(diff>1e-5 or it<10):
        diff=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k
        while(diffQTB>1e-8):
            for i in range(m):
                xpsa=P[ind2[i],:]
                r=y2[i]
                Qnew[i,:]=foo(r,xpsa,k,lq,Lq)   
            diffQTB=np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k
            Q=Qnew
        while(diffPSA>1e-8):
            for i in range(n):
                xqtb=Q[ind1[i],:]
                r=y1[i]
                Pnew[i,:]=foo(r,xqtb,k,lp,Lp)
            diffPSA=np.sum(np.multiply(Pnew-P,Pnew-P))/n/k+np.sum(np.multiply(Qnew-Q,Qnew-Q))/m/k
            P=Pnew
        it=it+1
    yhat_valid=np.sum(np.multiply((P[x_valid[:,0],:]),(Q[x_valid[:,1],:])),1)
    RMSE=sqrt((y_valid-yhat_valid)@(y_valid-yhat_valid)/y_valid.size)
    return RMSE,yhat_valid

2-layered gssvd rmse focused k =2,4,6,8,10

In [38]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
ls = Real(low=1e-7, high=1, prior='uniform',
                             name='ls')
la = Real(low=1e-7, high=1, prior='uniform',
                             name='la')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
lt = Real(low=1e-7, high=1, prior='uniform',
                             name='lt')
lb = Real(low=1e-7, high=1, prior='uniform',
                             name='lb')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Ls = Real(low=1e-7, high=1, prior='uniform',
                             name='Ls')
La = Real(low=1e-7, high=1, prior='uniform',
                             name='La')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
Lt = Real(low=1e-7, high=1, prior='uniform',
                             name='Lt')
Lb = Real(low=1e-6, high=1, prior='uniform',
                             name='Lb')
dimensions = [lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb]

In [39]:
@use_named_args(dimensions=dimensions)
def Fitness(lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb):
    print('lp:', lp)
    print()
    rmse,yhat = mygsm(mylasso,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,Shopgroup,Itemgroup,
                      n,m,5*k,lp,ls,la,lq,lt,lb,Lp,Ls,La,Lq,Lt,Lb)
    print()
    print("rmse:",rmse)
    print()
    return rmse

In [40]:
rmsev=np.zeros(shape=(3,5))
rmset=np.zeros(shape=(3,5))

In [41]:
start=time.time()
rmset=np.zeros(shape=(3,5))
rmsev=np.zeros(shape=(3,5))
for k in range(1,6):
    default_parameters = [1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3]
    search_result = gp_minimize(func=Fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    rmsev[2,k-1]=search_result.fun
    rmset[2,k-1]=mygsm(mylasso,ind1,y1,ind2,y2,x_train,x_test,y,y_test,Shopgroup,Itemgroup,n,m,5*k,para[0],
    para[1],
    para[2],
    para[3],
    para[4],
    para[5],
    para[6],
    para[7],
    para[8],
    para[9],
    para[10],
    para[11])[0]
print(time.time()-start)


lp: 0.001


rmse: 1.5471947432279785

lp: 0.9508187855487013


rmse: 1.5419509938995963

lp: 0.5367194847606038


rmse: 1.6783220275721336

lp: 0.8617864074848623


rmse: 1.6452007109973

lp: 0.5630264729645458


rmse: 1.6704565304760708

lp: 0.08464341300600847


rmse: 1.434935369605985

lp: 0.5027138994433967


rmse: 1.6645680244268597

lp: 0.9451897040396847


rmse: 1.669156582421218

lp: 0.18316498773558157


rmse: 1.6725551987772564

lp: 0.0714853986578972


rmse: 1.6725511260969192

lp: 0.6666027548926782


rmse: 1.299696317227699

lp: 0.378004777649255


rmse: 1.4188467982079163

lp: 1.0


rmse: 1.696688890733594

lp: 0.4752453876958103


rmse: 1.6699068951383778

lp: 0.9445120772673496


rmse: 8.004589785004981

lp: 0.5568860418437309


rmse: 1.6230631111093938

lp: 0.25839675150784647


rmse: 1.3727759057045081

lp: 0.8840290457131545


rmse: 1.550912208371857

lp: 0.7507396849471318


rmse: 1.2529343751905984

lp: 0.658409912443351


rmse: 1.654822163583317

lp: 0.23399097723

lp: 0.5185265343818133


rmse: 1.6574565231451492

lp: 0.05781678995864556


rmse: 1.5608003419906966

lp: 0.24206067319459237


rmse: 1.631483611496746

lp: 0.1804576095410182


rmse: 1.616931562346607

lp: 0.9362414884762778


rmse: 1.6420674399639585

lp: 0.7209119897518055


rmse: 1.6294186595087108

lp: 1e-07


rmse: 1.2085204063600672

lp: 0.10025258680932972


rmse: 1.6733122290400724

lp: 0.2236667256498986


rmse: 1.5462173457913966

lp: 0.2835334046556767


rmse: 1.6749547677454855

lp: 0.5263269927331862


rmse: 1.6622840538975365

lp: 0.5094248901414784


rmse: 1.661914542094615

lp: 0.3758529250296067


rmse: 1.6898022568096824

lp: 0.5412247056328281


rmse: 1.673656356351611

lp: 0.7640686018483865


rmse: 1.6764919578283817

lp: 0.43257709569790165


rmse: 1.3501320955756282

lp: 0.9305939509203885


rmse: 1.2283078314236273

lp: 0.3075555412322244


rmse: 1.524907295856546

lp: 0.9194215166335753


rmse: 1.206475477225371

lp: 0.9990386765795749


rmse: 1.4252132588876


rmse: 1.136865795970525

lp: 1e-07


rmse: 1.1858104743278393

lp: 0.3692390718005138


rmse: 1.0939705576283263

lp: 0.6102580280509288


rmse: 1.201937580534718

lp: 1e-07


rmse: 1.0813559517095968

lp: 0.9271003933826119


rmse: 1.2373101799380648

lp: 1e-07


rmse: 1.0928018312317898

lp: 0.48592587691148426


rmse: 1.2471593090700581

lp: 0.9016842568114581


rmse: 1.1479668394307798

lp: 1e-07


rmse: 1.1776648051296454

lp: 1e-07


rmse: 1.135726373774878

lp: 0.25678076105789566


rmse: 1.014985341102678

lp: 1e-07


rmse: 1.1916303644137216

lp: 0.14050028452958058


rmse: 1.2047684870554436

lp: 1e-07


rmse: 1.322796243573405

lp: 1e-07


rmse: 1.2406630852725173

lp: 0.7654064147796458


rmse: 1.3061759838584908

lp: 0.2683271737495841


rmse: 1.674705140651788

lp: 0.8763243756297133


rmse: 1.183715172247305

lp: 1e-07


rmse: 1.3036342847558045

lp: 1.0


rmse: 1.2740577888242188

lp: 0.059381481344615045


rmse: 1.2303725784052653

lp: 0.1585081901652363


rmse: 1.350

1-layered gssvd rmse focused 

In [42]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
ls = Real(low=1e-7, high=1, prior='uniform',
                             name='ls')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
lt = Real(low=1e-7, high=1, prior='uniform',
                             name='lt')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Ls = Real(low=1e-7, high=1, prior='uniform',
                             name='Ls')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
Lt = Real(low=1e-7, high=1, prior='uniform',
                             name='Lt')
dimensions = [lp,ls,lq,lt,Lp,Ls,Lq,Lt]

In [43]:
@use_named_args(dimensions=dimensions)
def Fitness(lp,ls,lq,lt,Lp,Ls,Lq,Lt):
    rmse,yhat = mygsm1(mylasso,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,Shopgroup,Itemgroup,
                      n,m,5*k,lp,ls,lq,lt,Lp,Ls,Lq,Lt)
    return rmse

In [44]:
start=time.time()
for k in range(1,6):
    default_parameters = [1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3,1e-3]
    search_result = gp_minimize(func=Fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=70,
                            x0=default_parameters)
    para=search_result.x
    rmsev[1,k-1]=search_result.fun
    rmset[1,k-1]=mygsm1(mylasso,ind1,y1,ind2,y2,x_train,x_test,y,y_test,Shopgroup,Itemgroup,n,m,5*k,para[0],
    para[1],
    para[2],
    para[3],
    para[4],
    para[5],
    para[6],
    para[7])[0]
print(time.time()-start)

12209.72963809967


nogroupsvd rmse focused

In [45]:
lp = Real(low=1e-7, high=1, prior='uniform',
                             name='lp')
lq = Real(low=1e-7, high=1, prior='uniform',
                             name='lq')
Lp = Real(low=1e-7, high=1, prior='uniform',
                             name='Lp')
Lq = Real(low=1e-7, high=1, prior='uniform',
                             name='Lq')
dimensions = [lp,lq,Lp,Lq]

In [46]:
@use_named_args(dimensions=dimensions)
def Fitness(lp,lq,Lp,Lq):
    print('lp:', lp)
    print()
    rmse,yhat = mygsm0(mylasso,ind1,y1,ind2,y2,x_train,x_valid,y,y_valid,
                      n,m,5*k,lp,lq,Lp,Lq)
    print()
    print("rmse:",rmse)
    print()
    return rmse

In [47]:
start=time.time()
for k in range(1,6):
    default_parameters = [1e-3,1e-3,1e-3,1e-3]
    search_result = gp_minimize(func=Fitness,
                            dimensions=dimensions,
                            acq_func='EI', 
                            n_calls=100,
                            x0=default_parameters)
    para=search_result.x
    rmsev[0,k-1]=search_result.fun
    rmset[0,k-1]=mygsm0(mylasso,ind1,y1,ind2,y2,x_train,x_test,y,y_test,n,m,5*k,para[0],
    para[1],
    para[2],
    para[3])[0]
print(time.time()-start)

lp: 0.001


rmse: 1.0046904153304264

lp: 0.44915776036827304


rmse: 1.684278598517065

lp: 0.6839474369872894


rmse: 1.684278598517065

lp: 0.9421340999047961


rmse: 1.684278598517065

lp: 0.08031047113054234


rmse: 1.4100802252705185

lp: 0.7742577263053388


rmse: 1.684278598517065

lp: 0.07883959479766763


rmse: 1.684278516688526

lp: 0.7169066554063526


rmse: 1.684278598517065

lp: 0.0728177494927146


rmse: 1.684278598517065

lp: 0.5168799068607054


rmse: 1.684278598517065

lp: 0.7649656896956556


rmse: 1.6116535848591134

lp: 0.14038337068129453


rmse: 1.684278598517065

lp: 1e-07


rmse: 1.683954082761868

lp: 1e-07


rmse: 1.684278598517065

lp: 0.7908552068643955


rmse: 1.0984626828998862

lp: 0.5515720314042253


rmse: 1.0917410895828223

lp: 0.4838745338413549


rmse: 1.1538528845649498

lp: 0.027352378779614946


rmse: 1.684278598517065

lp: 0.03601687126627124


rmse: 1.684278598517065

lp: 0.9220644733301071


rmse: 1.684278598517065

lp: 0.37719390570751177





rmse: 1.684278598517065

lp: 0.4856056204731885


rmse: 1.684278598517065

lp: 0.7469285610936477


rmse: 1.684278598517065

lp: 0.01787457378965853


rmse: 1.684278598517065

lp: 0.16942793166509837


rmse: 1.684278598517065

lp: 0.7942146243881562


rmse: 1.684278598517065

lp: 0.973340628199272


rmse: 1.684278598517065

lp: 0.6959821176375939


rmse: 1.684278598517065

lp: 0.16810193371320464


rmse: 1.684279914095108

lp: 0.7098109524840844


rmse: 1.684278598517065

lp: 0.35524561052828124


rmse: 1.684278598517065

lp: 0.9445216059695867


rmse: 1.684278598517065

lp: 0.6416632560608745


rmse: 1.684278598517065

lp: 0.9486999861533746


rmse: 1.684278598517065

lp: 0.12303487309535256


rmse: 1.684278598517065

lp: 0.32604306758119733


rmse: 1.684278598517065

lp: 0.5920864927279142


rmse: 1.6832493850359453

lp: 0.2965318792031632


rmse: 1.684278598517065

lp: 0.8473258108847973


rmse: 1.6842752779080037

lp: 0.8371571600138917


rmse: 1.684278598517065

lp: 0.63003754294


rmse: 1.684278598517065

lp: 0.5554613405616902


rmse: 1.684278598517065

lp: 0.7782270238452561


rmse: 1.684278598517065

lp: 0.8025507249703052


rmse: 1.684278598517065

lp: 1e-07


rmse: 1.684278598517065

lp: 1.0


rmse: 1.684278598517065

lp: 0.4874042830462904


rmse: 1.684278598517065

lp: 0.631714296282934


rmse: 1.684278598517065

lp: 1e-07


rmse: 1.684278598517065

lp: 0.8074534113193715


rmse: 1.684278598517065

lp: 0.8313823463349814


rmse: 1.612817126886467

lp: 1e-07


rmse: 1.684278598517065

lp: 1.0


rmse: 1.684278598517065

lp: 0.2072141976756916


rmse: 1.684278598517065

lp: 1e-07


rmse: 1.684278598517065

lp: 0.91468779621801


rmse: 1.684278598517065

lp: 0.39747678920878543


rmse: 1.684278598517065

lp: 0.6250948442649221


rmse: 1.684278598517065

lp: 0.8808397780236691


rmse: 1.684278598517065

lp: 0.4759463958088307


rmse: 1.7827032452866032

lp: 0.8284421649678686


rmse: 1.684278598517065

lp: 0.014305522047002809


rmse: 1.6768921664259588

lp: 

In [48]:
rmsev

array([[0.95666972, 1.01706695, 1.10193213, 1.05296574, 1.05796653],
       [1.06253953, 1.16406059, 1.12963083, 1.13064522, 1.07759259],
       [1.10748489, 1.0880455 , 1.11260372, 1.01498534, 1.07293984]])

In [49]:
rmset

array([[1.19224091, 1.01520053, 1.07035914, 1.20437472, 1.04570873],
       [1.29803953, 1.37422698, 1.18636503, 1.06428892, 1.12252359],
       [1.27384904, 1.05269498, 1.17570753, 1.06698745, 1.08432939]])

In [ ]:
pd.DataFrame(rmset).to_csv('rmsetdiff')
pd.DataFrame(rmsev).to_csv('rmsevdiff')